# Processing MCSL summer swimming result

In [72]:
from bs4 import BeautifulSoup
import csv
import os
import glob
import pandas as pd
import re

In [73]:
#file/folder
htmlbasepath = "./data/2023/week2"
filename_html = 'ARHvCLM.html'
filename_output = filename_html.replace('.html','.csv')

htmlpath = os.path.join(htmlbasepath, f'{filename_html}')
# print(path)

In [74]:
with open(htmlpath, 'r') as f:
    contents = f.read()
    soup = BeautifulSoup(contents, 'lxml')

# print(soup.prettify())

In [75]:
htmlfile_list = htmlbasepath.split('/')
year = htmlfile_list[2]
week = htmlfile_list[3]

In [76]:
table = soup.find("table")
cols = table.findAll("td")

content = []
result = []
record = []

for td in cols:
    try:
        td.string = td.get_text(strip=True)
        string = ''.join(td.find(string=True))
        content.append(string)
    except: pass


#loop the rest
for line in content:
    if line[0:5] == 'Event':
        substring = "AthleteSeedFinal"
        event = re.split(substring, line)[0]

    else:
        #start of new record, when it sees rank
        if (line[1:2] == '.' and len(line)==2) or (line[2:3] == '.' and len(line) == 3) or (line[2:3] == 'T' and len(line) == 4): #T is for tie
            if result:
                record.append(result)
                result = []
            result.append(year)
            result.append(week)
            result.append(event)
            result.append(line)
        else:
            result.append(line)



In [77]:
#take out relay record,
individual_record = []
for l in record:
    if 'Relay' not in l[2]: #third element on the list is where the event is located
        #print(l)|
        individual_record.append(l)
        

In [78]:
df = pd.DataFrame(individual_record, columns =['year', 'week', 'event', 'rank', 'swimmer', 'seed', 'final']) 

In [79]:
df['swimmer']

0       Sunjaya, Brian A (10) (ARH)
1            Tang, Kevin (12) (CLM)
2         Chen, Joseph S (10) (CLM)
3           Yi, Bradyn Y (11) (CLM)
4           Peter, Kidus (12) (ARH)
                   ...             
295          Kim, Conrad (15) (ARH)
296        Crown, Ella A (15) (CLM)
297       Lee, Juliana J (16) (CLM)
298    Stoudt, Kaitlyn E (17) (CLM)
299          Wong, Lydia (17) (ARH)
Name: swimmer, Length: 300, dtype: object

In [80]:
#special handling for swimmer with () on its name
df['swimmer'] = df['swimmer'].apply(lambda x:x.replace('(Dan)', '- Dan'))
df['swimmer'] = df['swimmer'].apply(lambda x:x.replace('(Ben)', '- Ben'))
df['swimmer'] = df['swimmer'].apply(lambda x:x.replace('(Jojo)', '- Jojo'))
df['swimmer'] = df['swimmer'].apply(lambda x:x.replace('(Jorie)', '- Jorie'))




#splitting swimmer colunn into 3
df[['swimmer_name', 'swimmer_age', 'swimmer_team']] = df['swimmer'].str.split(pat='(', expand=True, n=2)

# #dropping progoma; swimmer column
# del df['swimmer']

In [81]:
# #replace function
# def replace_enhanced(x):
#     if x is None:
#         return(x)
#     else:
#         x.replace(')','')
#         return(x)
    
#remove ')' from age and team
df['swimmer_age'] = df['swimmer_age'].apply(lambda x:x.replace(')', ''))
df['swimmer_team'] = df['swimmer_team'].apply(lambda x:x.replace(')', ''))


In [82]:
# #add seconds for seed and final
# x = '34.88'
# x_list = x.split(':')

# time_list = x.split(':')
# seconds = float(time_list[0])*60 + float(time_list[1])
# print(seconds)


def x_to_seconds(x):
    x = x.replace('X', '') #replacing exibition time
    x_list = x.split(':')
    if x in ('NT', 'NS', 'DQ', 'DNF'):  # ignore NT, NS, DQ, and DNF
        return(x)
    elif len(x_list) == 1: #only seconds no minutes
         return(float(x_list[0]))
    else:
        return(float(x_list[0])*60 + float(x_list[1]))
# x_to_seconds(x)


df['seed_seconds'] = df['seed'].apply(lambda x:x_to_seconds(x))
df['final_seconds'] = df['final'].apply(lambda x:x_to_seconds(x))



In [83]:
#get event number
df['event_no'] = df['event'].apply(lambda x:x.split('-')[0].replace('Event ', ''))


In [84]:
df.head(2).T

,0,1
year,2023,2023
week,week2,week2
event,Event 1 - Boys 12U 100M Individual Medley,Event 1 - Boys 12U 100M Individual Medley
rank,1.,2.
swimmer,"Sunjaya, Brian A (10) (ARH)","Tang, Kevin (12) (CLM)"
seed,1:19.19,1:29.38
final,1:18.09,1:28.70
swimmer_name,"Sunjaya, Brian A","Tang, Kevin"
swimmer_age,10,12
swimmer_team,ARH,CLM


In [85]:
df.tail()

,year,week,event,rank,swimmer,seed,final,swimmer_name,swimmer_age,swimmer_team,seed_seconds,final_seconds,event_no
295,2023,week2,Event 47 - Boys 15-18 50M Butterfly,6.,"Kim, Conrad (15) (ARH)",38.73,39.47,"Kim, Conrad",15,ARH,38.73,39.47,47
296,2023,week2,Event 48 - Girls 15-18 50M Butterfly,1.,"Crown, Ella A (15) (CLM)",31.62,32.78,"Crown, Ella A",15,CLM,31.62,32.78,48
297,2023,week2,Event 48 - Girls 15-18 50M Butterfly,2.,"Lee, Juliana J (16) (CLM)",33.01,34.27,"Lee, Juliana J",16,CLM,33.01,34.27,48
298,2023,week2,Event 48 - Girls 15-18 50M Butterfly,3.,"Stoudt, Kaitlyn E (17) (CLM)",33.06,34.81,"Stoudt, Kaitlyn E",17,CLM,33.06,34.81,48
299,2023,week2,Event 48 - Girls 15-18 50M Butterfly,4.,"Wong, Lydia (17) (ARH)",36.97,37.25,"Wong, Lydia",17,ARH,36.97,37.25,48


In [86]:
#export to csv with | delimited
df.to_csv(filename_output,sep='|', index=False) # Use pipe to seperate data


In [87]:
#counting swimmer by team/age
df.groupby(['swimmer_age', 'swimmer_team'])['swimmer_name'].count()

swimmer_age  swimmer_team
10           ARH             22
             CLM             19
11           ARH             19
             CLM             10
12           ARH             11
             CLM             15
13           ARH             26
             CLM             16
14           ARH             10
             CLM             19
15           ARH             21
             CLM              5
16           CLM             12
17           ARH              4
             CLM             13
18           CLM              4
6            ARH              3
             CLM              3
7            ARH             10
             CLM              6
8            ARH             14
             CLM             18
9            ARH             11
             CLM              9
Name: swimmer_name, dtype: int64